# Projeto 1 - Ciência dos Dados

Nome: Duilio Caoam Beneducci Moraes

Nome: Victoria Leal Garcia de Souza

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from emoji import UNICODE_EMOJI

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'dados.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo dados.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,"rt @dudacoutin_: vontade de comer brigadeiro, ...",1
1,qm q vai faze um brigadeiro branco p mim to co...,1
2,@vallnicolau tá agora eu to com vontade de com...,1
3,me deu uma vontade de comer brigadeiro com mor...,1
4,tava com vontade de comer abacaxi e brigadeiro...,1


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,relevancia
0,meu deus ea vontade d um brigadeiro pra hj,1
1,"to morrendo de vontade de um brigadeiro, mas s...",0
2,vontade de comer brigadeiro quase queimado,1
3,uma vontade imensa de comer brigadeiro tô nem ...,1
4,a vontade de comer brigadeiro tá grande,1


In [5]:
test.Teste.value_counts() #verificando se temos retweets

tá me dando vontade de fazer brigadeiro                                                                                                                                                     1
fiz brigadeiro e perdi a vontade de comer 🤪                                                                                                                                                 1
vontade de comer bolo de aniversário e me entupir de brigadeiro e beijinho                                                                                                                  1
vontade de fazer um brigadeiro de oreo                                                                                                                                                      1
vontade de comer um bolo de chocolate com bastante brigadeiro pqp tpm desgraçada 😩                                                                                                          1
                                                  

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Se tem a ver com querer brigadeiro, ou querer comprar brigadeiro, consideramos relevantes. "Não relevante" diz respeito aos tweets que não tinham nada sobre brigadeiro, ou que eram sobre fazer brigadeiro.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
quer=''
naoquer=''
for i in range(len(train.Relevancia)):
    if train.Relevancia[i] == 1:
        quer=quer+train.Treinamento[i]

for i in range(len(train.Relevancia)):
    if train.Relevancia[i] == 0:
        naoquer=naoquer+train.Treinamento[i]

In [7]:
import re 
def cleanup(text):
    #import string
    punctuation = '[!-.:?;,(\n)('')("")/@(rt)(\)]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def limpa(text): #separa emojis
    return ''.join(' ' + char if char in UNICODE_EMOJI else char for char in text).strip()

In [8]:
#limpa o texto
quer_limpo=limpa(cleanup(quer.lower()))
naoquer_limpo=limpa(cleanup(naoquer.lower()))

#tranforma em um conjunto de palavras
palavras_quer=quer_limpo.split()
palavras_naoquer=naoquer_limpo.split()

#gera uma planilha
serie_quer=pd.Series(palavras_quer)
serie_naoquer=pd.Series(palavras_naoquer)

#conta as quantidades
tabela_quer=serie_quer.value_counts()
tabela_naoquer=serie_naoquer.value_counts()

#faz as quantidades relativas
relativo_quer=serie_quer.value_counts(True)
relativo_naoquer=serie_naoquer.value_counts(True)

In [9]:
#definindo variaveis
probquer = len(serie_quer)/(len(serie_quer)+len(serie_naoquer))
probnaoquer = len(serie_naoquer)/(len(serie_quer)+len(serie_naoquer))

def classifica(x):
    #criando local onde vai ficar as varaiveis
    x.insert(2,"treinado",'')
    for k in range(len(x.Teste)):
        #gerando uma lista com os textos de cada célula
        x2=''
        x2=x.Teste[k]
        #limpando o texto
        x2=limpa(cleanup(x2.lower()))
        #dividindo o texto em palavras
        x3=x2.split()
        #resetando variaveis
        probFraseDadoq= 1
        probFraseDadon= 1
        #aqui mo trbalho está em cima de cada palavra
        for i in range(len(x3)):
            #verificando a probabilidade de estar no que quer o produto
            if x3[i] in relativo_quer:
                probFraseDadoq= probFraseDadoq *relativo_quer[x3[i]]
                #se estiver ali pode estar no outro tambem então verificamos
                if x3[i] in relativo_naoquer:
                    probFraseDadon=probFraseDadon *relativo_naoquer[x3[i]]
            #verificando se está apenas na probabilidade de não quer
            elif x3[i] in relativo_naoquer:
                probFraseDadon= probFraseDadon *relativo_naoquer[x3[i]]
            #multiplica pela probabilidade global
            probFraseDadoq=probFraseDadoq*probquer
            probFraseDadon=probFraseDadon*probnaoquer
        #define qual valor vai ser colocado na coluna criada
        if (probFraseDadoq>probFraseDadon):
            x.treinado[k]=1
        elif (probFraseDadoq<probFraseDadon):
            x.treinado[k]=0

classifica(test)
test.treinado.value_counts()

C:\Users\duili\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\duili\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1    83
0    76
Name: treinado, dtype: int64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [10]:
test.relevancia = test.relevancia.astype('int64')
test.treinado = test.treinado.astype('int64')

v1=test.relevancia
v2=test.treinado


In [11]:
#x é a relevancia y é o teste sempre.
def performance(x,y):
    #resetando variaveis
    vp=0
    vn=0
    fp=0
    fn=0
    
    for i in range(len(x)):
        #verdadeiros
        if x[i] == y[i]:
            #positivo
            if x[i] == 1:
                vp +=1
            #negativo
            elif x[i] == 0:
                vn +=1
        #falsos
        elif x[i] != y[i]:
            #positivo
            if x[i] == 1:
                fp +=1
            #negativo
            elif x[i] == 0:
                fn +=1
    #definindo os valores relativos
    n=vn+fn
    p=vp+fp
    vp=(vp/p)*100
    fp=(fp/p)*100
    vn=(vn/n)*100
    fn=(fn/n)*100
    
    print("a porcentagem de verdadeiros positisvos é de: ",vp)
    print("a porcentagem de Falsos positisvos é de: ",fp)
    print("a porcentagem de verdadeiros negativos é de: ",vn)
    print("a porcentagem de Falsos negativos é de: ",fn)
performance(v1,v2)

a porcentagem de verdadeiros positisvos é de:  55.434782608695656
a porcentagem de Falsos positisvos é de:  44.565217391304344
a porcentagem de verdadeiros negativos é de:  52.23880597014925
a porcentagem de Falsos negativos é de:  47.76119402985074


In [12]:
pd.crosstab(test.relevancia,test.treinado)

treinado,0,1
relevancia,,
0,35,32
1,41,51


In [13]:
#verificando se é correto
vn=(35/67)*100
fn=(32/67)*100
fp=(41/92)*100
vp=(51/92)*100
print(vn,fn,vp,fp)
#média do desemprenho
(vp+vn)/2

52.23880597014925 47.76119402985074 55.434782608695656 44.565217391304344


53.83679428942246

___
### Concluindo

Podemos concluir que o nosso classificador precisa de mais acurácia, pois sua performance ficou perto dos 50%. Precisariamos melhorar seu desempenho; uma forma de fazer isso seria criar maneiras de corrigir palavras erradas que acabam aparecendo apenas uma vez na base de dados, e assim atrapalha os valores relativos e absolutos.

Mensagens sarcásticas e de dupla negação não são identificadas assim, ou seja, o processamento de dados não sabe identificar isso, uma vez que o programa interpreta palavra por palavra, gerando uma maior probabilidade de falsos positivos e negativos.

Como futura expansão colocaríamos um corretor de texto, e geraria uma maior base de dados para o program, pois assim ele poderia reconhecer mais palavras para gerar um maior índice de acerto.

Não poderíamos alimentar a base de treinamento por si, pois primeiro a base de treinamento tem uma taxa relativamente baixa de acerto, e as palavras que não são reconhecidas ainda podem gerar conflitos na base de dados, onde uma palavra ainda não classificada pode gerar um erro para as antigas classificações.

Um cenário fora do contexto do projeto em que o classificador pode ser usado são as plataformas de streaming de música (como Spotify ou Deezer), onde um usuário pode escrever uma parte de uma música, e o classificador procuraria na sua base de dados a qual música ela pertence - claro, com uma base de dados muito maior que a nossa e com uma leitura condicional onde a primeira palavra deve estar associada a segunda e assim por diante, além de considerar erros ao lembrar a letra (trocar alguma palavra, por exemplo). Outro cenário seria para o uso em bibliotecas, onde por frases de um autor seria possível encontrar suas obras e/ou quais obras fazem referência àquele autor.

Uma forma de melhorar nosso classificador seria de passar palavra por palvra em um corretor de ortografia como o da Google (para isso poderíamos usar um código npl dentro de cada palavra), além de reconhecer conjuntos de palavras - não só palavras individuais (gerando bases de dados com duas variáveis em uma lista), assim possibilitando um erro menor por casos de sarcasmo e coloquialismos.

Outra iteração possível seria gerar classificações para derivados de brigadeiro. Como na base havia muita pipoca com brigadeiro e bolo de brigadeiro, isso poderia indicar qual o maior interesse e poderia portanto agir como uma classificação secundária dentro da relevância ou não de um produto simplesmente adicionando uma coluna com esses valores na base de dados. No entanto, isso implicaria em ter que reescrever a classificação, uma vez que descartamos as vontades de derivados do produto principal.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**